In [1]:
import pandas as pd
from datetime import datetime
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import time
import requests, bs4
import config
from sqlalchemy import create_engine

In [2]:
# reading in txt file from https://doi.org/10.7910/DVN/XPDSYQ
df = pd.read_csv('Stock_Trading_Data.txt', encoding='UTF-8')

In [3]:
# reformatting trade amount column values as $x,xxx.xx
df['trade_amount_max'] = df['trade_amount_max'].apply(lambda x: "${:,.2f}".format(x))
df['trade_amount_min'] = df['trade_amount_min'].apply(lambda x: "${:,.2f}".format(x))

In [4]:
# drop rows that contain NaN values in the ticker column
df = df.dropna(subset = ["ticker"])

In [5]:
# some ticker values were listed as "--" rather than NaN - removing those corresponding rows as well
df = df[df.ticker != "--"]

In [6]:
# create new column, combining min and max into a range
df["min_max"] = df["trade_amount_min"] + "-" + df["trade_amount_max"]

In [7]:
# reformatting values in the date column to datetime
for i in range(len(df)):
    day = df['trade_date'][i][:2]
    month = df['trade_date'][i][2:5]
    year = df['trade_date'][i][5:]
    date_full = month + "/" + day + "/" + year
    date_format = "%b/%d/%Y"
    df['trade_date'][i] = datetime.strptime(date_full, date_format)

/var/folders/q2/5v2xmgf12v118bg1jrms5ph80000gn/T/ipykernel_64885/269104711.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['trade_date'][i] = datetime.strptime(date_full, date_format)


KeyError: 44

In [8]:
# reformatting values in the party column from binary to descriptive text
for i in range(len(df)):
    if df.iloc[i,1] == 0:
        df.iloc[i,1] = "Republican"
    elif df.iloc[i,1] == 1.0:
        df.iloc[i,1] = "Democrat"
    elif df.iloc[i,1] == 2.0:
        df.iloc[i,1] = "Independent"

In [9]:
# reformatting values in the sex column from binary to descriptive text
for i in range(len(df)):
    if df.iloc[i,2] == 1:
        df.iloc[i,2] = "Male"
    elif df.iloc[i,2] == 0:
        df.iloc[i,2] = "Female"

In [10]:
# add new, empty "sector" column to dataframe
df["sector"] = ""

In [11]:
# resetting the dataframe to only contain the columns we need
df = df[[
    "senator",
    "party",
    "sex",
    "trade_date",
    "owner",
    "ticker",
    "sector",
    "type",
    "min_max",
    "one_month_sell_compare",
    "one_month_sell",
    "one_mo_vtsax_change",
    "buy_compare_one_year",
    "buy_one_year_change",
    "one_year_vtsax_change"
]]

In [12]:
# renaming columns
df = df.rename(columns={
    "trade_date":"dateoftrade",
    "type":"transactiontype",
    "min_max":"rangeoftransactionvalue",
    "one_month_sell_compare":"stockvaluevsmarket1month",
    "one_month_sell":"stockvaluechange1month",
    "one_mo_vtsax_change":"marketchange1month",
    "buy_compare_one_year":"stockvaluevsmarket1year",
    "buy_one_year_change":"stockvaluechange1year",
    "one_year_vtsax_change":"marketvaluechange1year"
})

In [13]:
# creating a list of only the unique tickers from the dataframe
unique_tickers = df["ticker"].unique()

In [14]:
# creating an empty list to hold sector values
sector_string = []
# looping through the unique tickers to go to that company's Yahoo Finance profile page
for ticker in unique_tickers:
    url = f"https://finance.yahoo.com/quote/{ticker}/profile?p={ticker}"
    headers = {"User-Agent":"Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    soup = bs4.BeautifulSoup(response.text, 'lxml')
    results = soup.find_all('p', class_='D(ib) Va(t)')
    # if no sector value on the page (like for funds), append the string "No sector" to the sector_string list
    if results == []:
        sector_string.append("No sector")
    # if there is a sector value, scrape and append that text to the sector_string list
    for result in results:
        sector = result.find("span", class_="Fw(600)").text
        sector_string.append(sector)

In [15]:
# create a dictionary using the unique ticker as the key and the sector information as the value
ticker_sector_dict = {unique_tickers[i]: sector_string[i] for i in range(len(unique_tickers))}

In [16]:
# using the dictionary created above to append the correct sector information in each row
ticker_list = df["ticker"].tolist()
sector_list = []
for i in range(len(ticker_list)):
    for key in ticker_sector_dict:
        if ticker_list[i] == key:
            sector_list.append(ticker_sector_dict[key])
df["sector"] = sector_list
df.head()

,senator,party,sex,dateoftrade,owner,ticker,sector,transactiontype,rangeoftransactionvalue,stockvaluevsmarket1month,stockvaluechange1month,marketchange1month,stockvaluevsmarket1year,stockvaluechange1year,marketvaluechange1year
0,Thomas Carper,Democrat,Male,2021-01-29 00:00:00,Spouse,XCAPX,No sector,Purchase,"$15,000.00-$50,000.00",NaN,NaN,0.031875,NaN,NaN,NaN
1,Patrick Toomey,Republican,Male,2021-01-28 00:00:00,Child,GME,Consumer Cyclical,Sale,"$1,000.00-$15,000.00",-0.454702,-0.438378,0.016324,NaN,NaN,NaN
2,Jerry Moran,Republican,Male,2021-01-27 00:00:00,Spouse,COST,Consumer Defensive,Sale,"$1,000.00-$15,000.00",-0.098641,-0.044025,0.054616,NaN,NaN,NaN
3,Jerry Moran,Republican,Male,2021-01-27 00:00:00,Self,COST,Consumer Defensive,Sale,"$1,000.00-$15,000.00",-0.098641,-0.044025,0.054616,NaN,NaN,NaN
4,Patrick Toomey,Republican,Male,2021-01-27 00:00:00,Child,SHOP,Technology,Sale,"$1,000.00-$15,000.00",0.136826,0.191442,0.054616,NaN,NaN,NaN


In [17]:
# !Must create seantors_db database before running this!
# Structure the database url from the config.py file and create the database engine
url = f'{config.user}:{config.password}@{config.hostname}:{config.port}/senatortrades_db'
engine = create_engine(f'postgresql://{url}', pool_pre_ping=True)

In [18]:
# After running schema.sql in your senators_db, check that the table exists
engine.table_names()

/var/folders/q2/5v2xmgf12v118bg1jrms5ph80000gn/T/ipykernel_64885/1597022047.py:2: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['trades']

In [ ]:
# Load the DataFrame to the database
df.to_sql(name='trades', con=engine, if_exists='append', index=False)